**QR Decomposition using householder Reflections**

Downloading matrix

In [1]:
import requests

# URL for the matrix on sparse.tamu.edu
matrix_id = "HB/bcsstk12"
url = f"https://sparse.tamu.edu/mat/{matrix_id}.mat"
dest_filename = "bcsstk12.mat"

# Download the file using requests
response = requests.get(url, stream=True)
response.raise_for_status() # Raise an exception for bad status codes

with open(dest_filename, 'wb') as f:
    for chunk in response.iter_content(chunk_size=8192):
        f.write(chunk)

print(f"Downloaded {dest_filename}")

Downloaded bcsstk12.mat


Loading Matrix

In [4]:
import numpy as np
from scipy.io import loadmat
from scipy.sparse import csr_matrix

# Load the .mat file
dat = loadmat("bcsstk12.mat")

# Access the matrix stored in the 'Problem' struct
Aqr= dat['Problem'][0, 0]['A']  # This is typically a sparse matrix already

# If A is not in CSR format, convert it
A_qr = csr_matrix(Aqr)

# Inspect matrix
print("Shape:", A_qr.shape)
print("Non-zeros:", A_qr.nnz)

Shape: (1473, 1473)
Non-zeros: 34241


4.QR Decomposition using householder Reflections

Matrix Requirement: Any shape (m x n)

In [3]:
import numpy as np

def qr_householder_cpu(A):
    A = np.array(A.todense(), dtype=float)
    m, n = A.shape
    Q = np.eye(m)
    R = A.copy()

    for k in range(n):
        x = R[k:, k]
        normx = np.linalg.norm(x)
        if normx == 0:
            continue

        e1 = np.zeros_like(x)
        e1[0] = 1.0
        v = x + np.sign(x[0]) * normx * e1
        v = v / np.linalg.norm(v)

        # Apply reflector to R
        R[k:, :] -= 2.0 * np.outer(v, v @ R[k:, :])

        # Apply reflector to Q
        Q[:, k:] -= 2.0 * np.outer(Q[:, k:] @ v, v)

    return Q, R

# Example usage
Q, R = qr_householder_cpu(A_qr)
A_dense = np.array(A_qr.todense(), dtype=float)
residual = np.linalg.norm(A_dense - Q @ R)
print("Q shape:", Q.shape)
print("R shape:", R.shape)
print("Top-left 5x5 block of Q:\n", Q[:5,:5])
print("Top-left 5x5 block of R:\n", R[:5,:5])
print("Residual norm ||A - QR||:", residual)

Q shape: (1473, 1473)
R shape: (1473, 1473)
Top-left 5x5 block of Q:
 [[-2.26967240e-01 -1.72972858e-02 -4.05005783e-03 -2.13939860e-02
  -5.34039468e-02]
 [-9.21930207e-01 -3.11262399e-01 -1.69079122e-02 -4.03073619e-03
  -1.11837349e-02]
 [ 1.00273856e-14  3.19953867e-14 -9.50341164e-01  6.27505244e-03
   1.68552046e-02]
 [ 6.99789438e-02 -2.07084015e-01  8.46128403e-04 -1.46655271e-01
  -8.01535397e-02]
 [ 2.65742825e-01 -7.86394992e-01  3.21314583e-03 -1.08269476e-01
  -3.56059659e-01]]
Top-left 5x5 block of R:
 [[-4.45813946e+06 -1.80791481e+07  3.39716842e+05  2.56298914e+06
   9.30834077e+06]
 [-8.37038201e-11 -3.88446785e+05  3.63768444e+04 -3.65687657e+06
  -1.26305916e+07]
 [ 6.33177009e-13 -9.79965537e-13 -1.91932604e+07 -6.36489644e+04
  -2.34130939e+05]
 [-3.43851841e-12  8.33041588e-13  3.52578125e-11 -9.63947816e+06
  -2.35773267e+06]
 [-3.09304092e-11  2.18856769e-12  9.87992128e-11 -2.32565774e-10
  -1.23231594e+07]]
Residual norm ||A - QR||: 3.4547366601790494e-06
